## 昇思+昇腾开发板：软硬结合玩转大模型实践能力认证（初级）

**环境准备：**

开发者拿到香橙派开发板后，首先需要进行硬件资源确认、镜像烧录以及CANN和MindSpore版本的升级，才可运行该案例，具体如下：

|**香橙派AIpro**|**镜像**|**CANN Toolkit/Kernels**|**MindSpore**|**MindSpore NLP**|
|:-------:|:-------:|:-------:|:-------:|:-------:|
|20T 24G|Ubuntu|8.0.0beta1|2.5.0|0.4分支|

- CANN检查与升级：参考[链接](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/orange_pi/environment_setup.html#3-cann%E5%8D%87%E7%BA%A7)
- MindSpore检查与升级：参考[链接](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/orange_pi/environment_setup.html#4-mindspore%E5%8D%87%E7%BA%A7)
- MindSpore NLP安装命令：
    ```bash
    pip install git+https://github.com/mindspore-lab/mindnlp.git@0.4
    ```

**场景说明：** 在本次实践中，我们将基于MindSpore NLP对DeepSeek-R1-Distill-Qwen-1.5B模型进行LoRA微调并推理，使得模型可以模仿《甄嬛传》中甄嬛的口吻进行对话。其中，LoRA（Low-Rank Adaptation）是一种参数高效微调（Parameter-Efficient Fine-Tuning, PEFT）方法。其核心思想是冻结原始网络参数，对Attention层中QKV等模块添加旁支。旁支包含两个低维度的矩阵A和矩阵B，微调过程中仅更新A、B 矩阵。通过这种方式，显著降低计算和内存成本，同时达到与全参数微调相近的效果。

模型链接：https://modelers.cn/models/MindSpore-Lab/DeepSeek-R1-Distill-Qwen-1.5B-FP16

**考核目标：** 本次实践旨在考核基于MindSpore NLP对大模型微调推理流程的掌握，共6个考点。考生需补全空缺处的代码，保证执行脚本全流程跑通。


In [ ]:
import os

import mindspore
import mindnlp
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer
from mindnlp.engine import TrainingArguments, Trainer
from mindnlp.dataset import load_dataset
from mindnlp.transformers import GenerationConfig
from mindnlp.peft import LoraConfig, TaskType, get_peft_model, PeftModel

from mindnlp.engine.utils import PREFIX_CHECKPOINT_DIR
from mindnlp.configs import SAFE_WEIGHTS_NAME
from mindnlp.engine.callbacks import TrainerCallback, TrainerState, TrainerControl

from mindspore._c_expression import disable_multi_thread
disable_multi_thread()

# 开启同步，用于定位问题，调试完毕后建议关闭同步
# mindspore.set_context(pynative_synchronize=True)

### 获取并加载数据集

本次实践使用了huanhuan数据集，该数据集从《甄嬛传》的剧本进行整理，从原始文本中提取出我们关注的角色的对话，并形成 QA 问答对，最终整理为json格式的数据，数据样本示例如下：

```text
[
    {
        "instruction": "小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——",
        "input": "",
        "output": "嘘——都说许愿说破是不灵的。"
    },
]
```

使用`openmind_hub`接口下载数据集，并通过`load_dataset`加载。

数据集链接：https://modelers.cn/datasets/MindSpore-Lab/huanhuan

In [ ]:
%pip install openmind_hub

In [ ]:
from openmind_hub import om_hub_download

# 从魔乐社区下载数据集
om_hub_download(
    repo_id="MindSpore-Lab/huanhuan",
    repo_type="dataset",
    filename="huanhuan.json",
    local_dir="./",
)

# 加载数据集
dataset = load_dataset(path="json", data_files="./huanhuan.json")

### 实例化tokenizer

创建一个分词器`tokenizer`，并配置其填充标记和填充位置。

In [ ]:
# 实例化tokenizer
tokenizer = AutoTokenizer.from_pretrained("MindSpore-Lab/DeepSeek-R1-Distill-Qwen-1.5B-FP16", mirror="modelers", use_fast=False, ms_type=mindspore.float16)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

### **考点1：数据处理**

`process_func`函数将原始的对话数据转换为适合模型微调的格式，后续为节约时间，使用`take`接口对数据集进行裁剪。

**要求：** 请将定义好的数据预处理函数应用于数据集`dataset`上，处理后的数据集保存在`formatted_dataset`中，并打印预处理后的数据，期望打印结果如下：
```text
User: 小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——

Assistant: 嘘——都说许愿说破是不灵的。<｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜><｜end▁of▁sentence｜>
```

**参考文档：** 具体实现参考MindSpore官网2.5.0版本学习-教程-快速上手（数据加载与处理）中的数据变换，或者参考[昇思+昇腾开发板：软硬结合玩转DeepSeek开发实战课程](https://www.hiascend.com/developer/courses/detail/1925362775376744449)

In [ ]:
# 定义数据处理逻辑
def process_func(instruction, input, output):
    MAX_SEQ_LENGTH = 64  # 最长序列长度
    input_ids, attention_mask, labels = [], [], []
    # 首先生成user和assistant的对话模板
    # User: instruction + input
    # Assistant: output
    formatted_instruction = tokenizer(f"User: {instruction}{input}\n\n", add_special_tokens=False)
    formatted_response = tokenizer(f"Assistant: {output}", add_special_tokens=False)
    # 最后添加 eos token，在deepseek-r1-distill-qwen的词表中， eos_token 和 pad_token 对应同一个token
    # User: instruction + input \n\n Assistant: output + eos_token
    input_ids = formatted_instruction["input_ids"] + formatted_response["input_ids"] + [tokenizer.pad_token_id]
    # 注意相应
    attention_mask = formatted_instruction["attention_mask"] + formatted_response["attention_mask"] + [1]
    labels = [-100] * len(formatted_instruction["input_ids"]) + formatted_response["input_ids"] + [tokenizer.pad_token_id]

    if len(input_ids) > MAX_SEQ_LENGTH:
        input_ids = input_ids[:MAX_SEQ_LENGTH]
        attention_mask = attention_mask[:MAX_SEQ_LENGTH]
        labels = labels[:MAX_SEQ_LENGTH]

    # 填充到最大长度
    padding_length = MAX_SEQ_LENGTH - len(input_ids)
    input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
    attention_mask = attention_mask + [0] * padding_length  # 填充的 attention_mask 为 0
    labels = labels + [-100] * padding_length  # 填充的 label 为 -100
    
    return input_ids, attention_mask, labels

# >>>>>>> 题目：将定义好的预处理函数应用于数据集上 <<<<<<<
formatted_dataset = ________

# 查看预处理后的数据
for input_ids, attention_mask, labels in formatted_dataset.create_tuple_iterator():
    print(tokenizer.decode(input_ids))
    break

# 为节约时间，将数据集裁剪
truncated_dataset = formatted_dataset.take(3)

### **考点2：LoRA配置**

加载预训练模型权重和生成配置，并通过`LoraConfig`配置LoRA参数。

**要求：** 请补齐LoRA配置中的Lora秩和Lora alpha，使得考点3中获取到的参与训练的参数量不超过总参数量的0.52%。

In [ ]:
model_id = "MindSpore-Lab/DeepSeek-R1-Distill-Qwen-1.5B-FP16"

base_model = AutoModelForCausalLM.from_pretrained(model_id, mirror="modelers", ms_dtype=mindspore.float16)
base_model.generation_config = GenerationConfig.from_pretrained(model_id, mirror="modelers")

base_model.generation_config.pad_token_id = base_model.generation_config.eos_token_id

# >>>>>>> 题目：LoRA配置，补齐Lora秩、Lora alpha <<<<<<<

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=________, # Lora 秩
    lora_alpha=________, # Lora alpha，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)

### **考点3：实例化LoRA模型**

实例化LoRA模型，打印训练参数量占比，并定义回调类`SavePeftModelCallback`，保存训练过程中的lora adapter权重。

**要求：** 请使用已有的MindSpore NLP接口`get_peft_model`，加载LoRA配置，实例化LoRA模型。

**参考文档：** https://github.com/mindspore-lab/mindnlp/blob/0.4/docs/en/tutorials/peft.md

In [ ]:
# >>>>>>> 题目：实例化LoRA模型 <<<<<<<
model = ________

# 获取模型训练参数占比数，发现仅占总参数量的0.516%
total_params = 0
lora_params = 0
for param in model.trainable_params():
    lora_params += param.size
for param in model.get_parameters():
    total_params += param.size
print('proportion of parameters: ', lora_params / total_params)

class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(
            args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}"
        )       

        # 保存adapter weights
        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        # 保存训练过程中的lora adapter权重
        kwargs["model"].save_pretrained(peft_model_path, safe_serialization=True)

        # 删除base model的saeftensors权重，节约更多空间
        base_model_path = os.path.join(checkpoint_folder, SAFE_WEIGHTS_NAME)
        os.remove(base_model_path) if os.path.exists(base_model_path) else None

        return control


### 启动微调

配置微调超参，并执行微调。执行完微调后，可在`./output/DeepSeek-R1-Distill-Qwen-1.5B`中找到`checkpoint-3`的文件夹，内有保存微调后的LoRA adapter权重。

In [ ]:
args = TrainingArguments(
    output_dir="./output/DeepSeek-R1-Distill-Qwen-1.5B",
    per_device_train_batch_size=1,
    logging_steps=1,
    num_train_epochs=1,
    save_steps=3,
    learning_rate=1e-4,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=truncated_dataset,
    callbacks=[SavePeftModelCallback],
)

trainer.train()

### 模型推理

In [ ]:
%pip install gradio==4.44.0

In [ ]:
import gradio as gr
import mindspore
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer
from mindnlp.transformers import TextIteratorStreamer
from threading import Thread

### **考点4：模型实例化**

加载tokenizer和预训练模型model，并在model的基础上使用PeftModel加载微调后的LoRA adapter权重。

**要求：** 请**使用MindSpore NLP的API接口**`AutoModelForCausalLM`和`AutoTokenizer`，实例化`tokenizer`和模型`model`，镜像地址为`modelers`，模型ID为`MindSpore-Lab/DeepSeek-R1-Distill-Qwen-1.5B-FP16`，数据类型为`float16`

**参考文档：** https://github.com/mindspore-lab/mindnlp/blob/0.4/docs/en/tutorials/quick_start.md

In [ ]:
# >>>>>>> 题目：实例化tokenizer和模型，镜像地址为modelers，模型ID为"MindSpore-Lab/DeepSeek-R1-Distill-Qwen-1.5B-FP16"，数据类型为float16 <<<<<<<
# >>>>>>> 补全实例化tokenizer和模型的代码 <<<<<<<
tokenizer = ________
model = ________
model = PeftModel.from_pretrained(model, "./output/DeepSeek-R1-Distill-Qwen-1.5B/checkpoint-3/adapter_model/")

### **考点5：构建输入对话**

`build_input_from_chat_history`函数用于根据聊天历史记录和当前消息构建一个消息列表，将聊天历史和当前消息整合成一个特定格式的列表，以便后续处理或传递给聊天机器人模型。

**要求：** 请补齐`build_input_from_chat_history`函数中对话历史记录的输入格式，即字典中`content`对应的内容

In [ ]:
system_prompt = "You are a helpful and friendly chatbot"

def build_input_from_chat_history(chat_history, msg: str):
    messages = [{'role': 'system', 'content': system_prompt}]
    for user_msg, ai_msg in chat_history:
        # >>>>>>> 题目：补齐对话历史记录的输入格式，即字典中“content”对应的内容 <<<<<<<
        messages.append({'role': 'user', 'content': ________})
        messages.append({'role': 'assistant', 'content': ________})
    messages.append({'role': 'user', 'content': msg})
    return messages

### **考点6：设置generate参数**

`predict`函数用于预测生成文本，它接收用户的消息和对话历史，然后生成一个回应，其中`generate_kwargs`包含生成文本所需的参数。

**要求：** 请补齐`predict`函数中`generate_kwargs`的参数，包括最大生成长度、top-p采样参数、重复惩罚系数

In [ ]:
# 预测生成文本
def predict(message, history):
    history_transformer_format = history + [[message, ""]]

    # 构建输入消息列表
    messages = build_input_from_chat_history(history, message)
    input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="ms",
            tokenize=True
        )
    streamer = TextIteratorStreamer(tokenizer, timeout=300, skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        input_ids=input_ids,
        streamer=streamer,
        # >>>>>>> 题目：设置最大生成长度 <<<<<<<
        max_new_tokens=________,
        do_sample=True,
        # >>>>>>> 题目：设置top-p采样参数 <<<<<<<
        top_p=________,
        temperature=0.1,
        num_beams=1,
        # >>>>>>> 题目：设置重复惩罚系数 <<<<<<<
        repetition_penalty=________
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()  # 在单独的线程中执行生成
    partial_message = ""
    for new_token in streamer:
        partial_message += new_token
        if '</s>' in partial_message:  # 如果出现结束标记，则终止循环
            break
        yield partial_message


### 启动Gradio聊天界面

创建一个简单的聊天机器人界面，用于对话问答。

In [ ]:
# 设置 Gradio 聊天界面
gr.ChatInterface(predict,
                 title="DeepSeek-R1-Distill-Qwen-1.5B",
                 description="问几个问题",
                 examples=['你是谁？', '你能做什么？']
                 ).launch()  # 启动 Web 界面